#Instalar librerías

In [3]:
# Instalar Roboflow para cargar datasets
!pip install roboflow
!pip install tensorflow==2.17.0


In [5]:
import keras
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
print(keras.__version__)
print(tf.__version__)

3.5.0
2.17.0


# Cargar Dataset de RoboFlow

In [6]:
from roboflow import Roboflow
rf = Roboflow(api_key="mlT63rhUs7OKHz2LKNMC")
project = rf.workspace("tattoo-d6iux").project("tattoo_types")
version = project.version(3)
dataset = version.download("multiclass")


loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to Tattoo_Types-3 in multiclass:: 100%|██████████| 226/226 [00:00<00:00, 2967.33it/s]


In [7]:
import keras
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
print(keras.__version__)
print(tf.__version__)

3.5.0
2.17.0


# Preparar los datos

In [8]:
# VALIDAR SI LAS CARPETAS EXISTEN
import os

def count_images_in_directory(directory):
    images = []
    for root, _, files in os.walk(directory):
        for file in files:
            if file.lower().endswith(('.png', '.jpg', '.jpeg', '.gif', '.bmp')):
                full_path = os.path.join(root, file)
                images.append(full_path)
    print(f"Total images in {directory}: {len(images)}")
    print(images)
    return images

print("Training images:")
a = count_images_in_directory("/content/Tattoo_Types-3/train")

print("\nValidation images:")
b = count_images_in_directory("/content/Tattoo_Types-3/valid")

Training images:
Total images in /content/Tattoo_Types-3/train: 186
['/content/Tattoo_Types-3/train/descarga-3-_jpg.rf.d0c4e1b71ffcd6c0eaf41e1ac01c61ba.jpg', '/content/Tattoo_Types-3/train/descarga-6-_jpg.rf.4085bff804dc30b31cb80ae3aecd8ed6.jpg', '/content/Tattoo_Types-3/train/cf34b84f8d3c6079ac3450bb8537539f_jpg.rf.14856412dba052cd5158021fa2edea16.jpg', '/content/Tattoo_Types-3/train/descarga-4-_jpg.rf.deb173a489f58f97d4c3bb0621938dec.jpg', '/content/Tattoo_Types-3/train/images-1-_jpg.rf.4aab9c3055b34f0cd46167d96276b990.jpg', '/content/Tattoo_Types-3/train/tatuaje_Logia_Barcelona_Julio_Voltaje_Aguila_02_jpg.rf.da29d8ebf59606d24f9b4a78d5882e92.jpg', '/content/Tattoo_Types-3/train/descarga-1-_jpg.rf.984fbd0af98d1bbed342188d5c3e6fb6.jpg', '/content/Tattoo_Types-3/train/descarga_jpg.rf.71cf054d123c0a6456cbd82b46d97698.jpg', '/content/Tattoo_Types-3/train/descarga-10-_jpg.rf.fa8d4ec7a8bcf17b7288c2295de04cf9.jpg', '/content/Tattoo_Types-3/train/images_jpg.rf.6ef6adefc55b295d6b3ed3bb43de487f

# Organizar las imagenes de la carpeta de train

In [9]:
import os
import shutil
import pandas as pd

# Ruta al archivo CSV y a la carpeta de imágenes
csv_path = "/content/Tattoo_Types-3/train/_classes.csv"
source_dir = a  # Lista con las rutas de las imágenes

# Cargar el archivo CSV
data = pd.read_csv(csv_path)

# Limpiar nombres de columnas eliminando espacios iniciales si los hay
data.columns = data.columns.str.strip()

# Crear un directorio de destino para las imágenes organizadas
train_dir = 'train_org'
os.makedirs(train_dir, exist_ok=True)

# Procesar cada fila del CSV
for _, row in data.iterrows():
    filename = row['filename']
    # Determinar la clase a partir de las columnas binarias
    classes = row[1:]  # Todas las columnas excepto 'filename'
    target_classes = classes[classes == 1].index.tolist()

    # Validar si existe al menos una clase para la imagen
    if not target_classes:
        print(f"No se encontró clase para la imagen {filename}.")
        continue

    # Buscar la ruta completa de la imagen en la lista de rutas
    image_path = next((path for path in source_dir if filename in path), None)

    if not image_path:
        print(f"No se encontró la imagen {filename} en la lista.")
        continue

    # Mover o copiar la imagen a la carpeta de cada clase correspondiente
    for cls in target_classes:
        class_dir = os.path.join(train_dir, cls)
        os.makedirs(class_dir, exist_ok=True)  # Crear la carpeta si no existe

        dest_path = os.path.join(class_dir, os.path.basename(image_path))
        shutil.copy(image_path, dest_path)  # Cambia `copy` a `move` si deseas mover
        print(f"Imagen {filename} copiada a {class_dir}.")

Imagen descarga-6-_jpg.rf.002e1900d75b492906810671588d8e2a.jpg copiada a train_org/Neotradicional.
Imagen descarga-6-_jpg.rf.00bf6ba8e40de5a1a9ebed46a2e1a1e2.jpg copiada a train_org/Neotradicional.
Imagen descarga-1-_jpg.rf.0380ab95dcbd03c89babe1c1f875af42.jpg copiada a train_org/Old-School.
Imagen images-1-_jpg.rf.00030ba64f621878f9b3193761cd25ca.jpg copiada a train_org/Old-School.
Imagen 7735e49e88b9f354c4827a93c519c12f_jpg.rf.08991ea879710c14b1d464ce1c95733c.jpg copiada a train_org/Old-School.
Imagen descarga-1-_jpg.rf.0c065cf345947525a6ffacaf3b57ec0d.jpg copiada a train_org/Blackwork.
Imagen descarga_jpg.rf.092610ccbc89a1cb9d9de624c1e57cdd.jpg copiada a train_org/Dotwork.
Imagen images-1-_jpg.rf.06392eb9ca6cf8807eb5e7ccbb3596a1.jpg copiada a train_org/Dotwork.
Imagen descarga_jpg.rf.05e2d6a292da981a3310cbbe0edc4f23.jpg copiada a train_org/Blackwork.
Imagen 6fe030da732b96912ff30d59716718e9_jpg.rf.0d253e5d5e9771d1d1286e12e14586f3.jpg copiada a train_org/Old-School.
Imagen 62e7e6ae904

# Organizar carpeta de valid por clase

In [11]:
import os
import shutil
import pandas as pd

# Ruta al archivo CSV y a la carpeta de imágenes
csv_path = "/content/Tattoo_Types-3/valid/_classes.csv"
source_dir = b  # Lista con las rutas de las imágenes

# Cargar el archivo CSV
data = pd.read_csv(csv_path)

# Limpiar nombres de columnas eliminando espacios iniciales si los hay
data.columns = data.columns.str.strip()

# Crear un directorio de destino para las imágenes organizadas
train_dir = 'valid_org'
os.makedirs(train_dir, exist_ok=True)

# Procesar cada fila del CSV
for _, row in data.iterrows():
    filename = row['filename']
    # Determinar la clase a partir de las columnas binarias
    classes = row[1:]  # Todas las columnas excepto 'filename'
    target_classes = classes[classes == 1].index.tolist()

    # Validar si existe al menos una clase para la imagen
    if not target_classes:
        print(f"No se encontró clase para la imagen {filename}.")
        continue

    # Buscar la ruta completa de la imagen en la lista de rutas
    image_path = next((path for path in source_dir if filename in path), None)

    if not image_path:
        print(f"No se encontró la imagen {filename} en la lista.")
        continue

    # Mover o copiar la imagen a la carpeta de cada clase correspondiente
    for cls in target_classes:
        class_dir = os.path.join(train_dir, cls)
        os.makedirs(class_dir, exist_ok=True)  # Crear la carpeta si no existe

        dest_path = os.path.join(class_dir, os.path.basename(image_path))
        shutil.copy(image_path, dest_path)  # Cambia `copy` a `move` si deseas mover
        print(f"Imagen {filename} copiada a {class_dir}.")

Imagen c4ebb0c54ff2f03d761a42bce58ddaec_jpg.rf.1d119081b22821ce3ab528cfb1130eea.jpg copiada a valid_org/Old-School.
Imagen 2b7b7a4fef9c9ae882bedab07e73ff15_jpg.rf.1f84126a39f8e5b493dd4f7915bb8d38.jpg copiada a valid_org/Old-School.
Imagen b7d7a06b1cfcb6c46cb9e8b90a412296_jpg.rf.12355925fa7c58f7716d54e369e10ba0.jpg copiada a valid_org/Old-School.
Imagen descarga-9-_jpg.rf.1debaa1b5de304dc569b557c4ee428fb.jpg copiada a valid_org/Blackwork.
Imagen descarga-11-_jpg.rf.1e1f72643470965d371e025cb14e6f79.jpg copiada a valid_org/Neotradicional.
Imagen descarga-2-_jpg.rf.237485647dc97dace577fa77b184bfd6.jpg copiada a valid_org/Dotwork.
Imagen descarga_jpg.rf.38685595e348eb380a6da2778d669b5d.jpg copiada a valid_org/Acuarela.
Imagen 2e2af2e3bb619ac98c347f8c1ab791c3_jpg.rf.2b59cb07c47c7458f68977445a64834e.jpg copiada a valid_org/Old-School.
Imagen images-3-_jpg.rf.38d347e288609fca48a643a205e08bd1.jpg copiada a valid_org/Old-School.
Imagen descarga-9-_jpg.rf.4c0c2952b77ecccdd192b291d0901f1d.jpg copi

# Entrenar el modelo

In [12]:
import pandas as pd
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D

# Directorios de entrenamiento y validación
train_dir = "/content/train_org"
valid_dir = "/content/valid_org"

# Crear el generador de datos
train_datagen = ImageDataGenerator(
    rescale=1.0/255,          # Escalar valores de píxeles entre 0 y 1
    rotation_range=20,        # Rotación aleatoria
    width_shift_range=0.2,    # Desplazamiento horizontal
    height_shift_range=0.2,   # Desplazamiento vertical
    shear_range=0.2,          # Deformación en corte
    zoom_range=0.2,           # Zoom aleatorio
    horizontal_flip=True,     # Volteo horizontal
    fill_mode="nearest"       # Rellenar píxeles vacíos
)

valid_datagen = ImageDataGenerator(rescale=1.0/255)

# Generador para el conjunto de entrenamiento (desde carpetas organizadas)
train_generator = train_datagen.flow_from_directory(
    directory=train_dir,
    target_size=(128, 128),    # Tamaño de las imágenes
    batch_size=32,             # Tamaño del lote
    class_mode="categorical"   # Clasificación multicategoría
)

# Generador para el conjunto de validación (desde carpetas organizadas)
valid_generator = valid_datagen.flow_from_directory(
    directory=valid_dir,
    target_size=(128, 128),    # Tamaño de las imágenes
    batch_size=32,             # Tamaño del lote
    class_mode="categorical"   # Clasificación multicategoría
)

# Crear el modelo de red neuronal
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(train_generator.num_classes, activation='softmax')  # Número de clases dinámico
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Entrenar el modelo
history = model.fit(
    train_generator,
    epochs=10,                   # Número de épocas
    validation_data=valid_generator
)


Found 189 images belonging to 5 classes.
Found 25 images belonging to 5 classes.


Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.


Epoch 1/10


Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.


6/6 ━━━━━━━━━━━━━━━━━━━━ 9s 844ms/step - accuracy: 0.2580 - loss: 2.7382 - val_accuracy: 0.1200 - val_loss: 1.6855
Epoch 2/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 7s 1s/step - accuracy: 0.3585 - loss: 1.5232 - val_accuracy: 0.3200 - val_loss: 1.5481
Epoch 3/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 9s 770ms/step - accuracy: 0.4404 - loss: 1.3974 - val_accuracy: 0.3200 - val_loss: 1.7551
Epoch 4/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 8s 1s/step - accuracy: 0.4328 - loss: 1.3429 - val_accuracy: 0.3200 - val_loss: 1.5990
Epoch 5/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 6s 775ms/step - accuracy: 0.4159 - loss: 1.3313 - val_accuracy: 0.3200 - val_loss: 1.6014
Epoch 6/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 10s 789ms/step - accuracy: 0.4094 - loss: 1.3228 - val_accuracy: 0.3200 - val_loss: 1.5960
Epoch 7/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 11s 776ms/step - accuracy: 0.4765 - loss: 1.1786 - val_accuracy: 0.3200 - val_loss: 1.4998
Epoch 8/10
6/6 ━━━━━━━━━━━━━━━━━━━━ 6s 884ms/step - accuracy: 0.4790 - loss: 1.1382 - val_accuracy: 0.3200 - val_loss: 1.7414
Epoch 9

# Probando el modelo

In [17]:
from tensorflow.keras.preprocessing import image
import numpy as np

# Ruta de la imagen que quieres probar
img_path = "/content/tradi.JPG"

# Cargar la imagen y redimensionarla al tamaño esperado por el modelo
img = image.load_img(img_path, target_size=(128, 128))  # Debe ser el mismo tamaño de entrada de tu modelo

# Convertir la imagen a un array
img_array = image.img_to_array(img)

# Escalar los valores de píxeles (de 0-255 a 0-1) de acuerdo con el preprocesamiento del ImageDataGenerator
img_array = img_array / 255.0

# Añadir una dimensión para que sea compatible con la entrada del modelo (1, 128, 128, 3)
img_array = np.expand_dims(img_array, axis=0)

# Hacer la predicción con el modelo entrenado (modelo ya cargado)
predictions = model.predict(img_array)

# Obtener la clase con la mayor probabilidad
predicted_class = np.argmax(predictions, axis=1)

# Mostrar las clases y sus índices
print("Clases y sus índices:", train_generator.class_indices)

# Imprimir la clase predicha
classes = list(train_generator.class_indices.keys())  # Lista de clases desde el generador
print("Clase detectada:", classes[predicted_class[0]])


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
Clases y sus índices: {'Acuarela': 0, 'Blackwork': 1, 'Dotwork': 2, 'Neotradicional': 3, 'Old-School': 4}
Clase detectada: Old-School
